In [1]:

import glob, sys, sqlite3

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from colorama import Fore, Back, Style
from matplotlib.ticker import PercentFormatter
from matplotlib.pyplot import figure

# Noms de colonnes à utiliser
names = ['Code Domaine', 'Domaine', 'Code Pays', 'Pays', 'Code Élément', 'Élément', 'Code Produit',
         'Produit', 'Code Année', 'Année', 'Unité', 'Valeur', 'Symbole', 'Description du Symbole']

fns = glob.glob('*/*.csv')
conn = sqlite3.connect('bdd.sqlite')

# Dictionnaires des csv
dfs = {}

for fn in fns:

    fns = fn.split('/')[-1][13:-4]

    if fns == 'Groupe_de_pays':
        df = pd.read_csv(fn)
    else:
        df = pd.read_csv(fn, names=names, header=0)

    print(f" - {fn:50s} ({df.shape[0]:6d}, {df.shape[1]:2d}) - {Fore.BLACK}{Back.GREEN}{fns}{Style.RESET_ALL}")
    
    # Récupération des données dans un sql
    df.to_sql(fns, con=conn, if_exists='replace')
    # Et dans le dictionnaire
    dfs[fns] = df

plt.rcParams["figure.figsize"] = (22,10)


 - data/FAOSTAT_2013_vegetal.csv                      (104871, 14) - vegetal
 - data/FAOSTAT_2013_animal.csv                       ( 37166, 14) - animal
 - data/FAOSTAT_2013_superficie.csv                   (   238, 14) - superficie
 - data/FAOSTAT_2023_Groupe_de_pays.csv               (  1941,  7) - Groupe_de_pays
 - data/FAOSTAT_2013_population.csv                   (   175, 14) - population
 - data/FAOSTAT_2013_sous_alimentation.csv            (   204, 14) - sous_alimentation
 - data/FAOSTAT_2013_cereal.csv                       ( 16057, 14) - cereal


# Question 1

In [2]:

# Récupèration des noms de pays et de la population
df_pop = pd.DataFrame(dfs['population'])[['Pays', 'Valeur']]

# On fait la somme
print(df_pop['Valeur'].sum() * 1000)
# Le résultat est erroné car la Chine est représenté plusieurs fois

# On sélectionne tout les doublons de la Chine commençant par "China, "
df_china = df_pop[df_pop['Pays'].str.startswith('China,')]

# On les drop
df_pop.drop(df_china.index, inplace=True)

nbr_total_humain = df_pop['Valeur'].sum() * 1000
print(nbr_total_humain)


8413993000
6997326000


# Question 2

In [3]:

# prod + imp - exp + var = disp = ali + sem + per + trai + aut + nou

df = dfs['vegetal']

# On récupère pour l'exemple du Blé en France
df = df.query("Pays == 'France' and Produit == 'Blé'")[['Élément', 'Valeur']].set_index('Élément')

prod = df.at['Production', 'Valeur']
imp  = df.at['Importations - Quantité', 'Valeur']
exp  = df.at['Exportations - Quantité', 'Valeur']
var  = df.at['Variation de stock', 'Valeur']
disp = df.at['Disponibilité intérieure', 'Valeur']
ali  = df.at['Aliments pour animaux', 'Valeur']
sem  = df.at['Semences', 'Valeur']
per  = df.at['Pertes', 'Valeur']
trai = df.at['Traitement', 'Valeur']
aut  = df.at['Autres Utilisations', 'Valeur']
nou  = df.at['Nourriture', 'Valeur']

print("prod + imp - exp + var             =", prod + imp - exp + var)
print("disp                               =", disp)
print("ali + sem + per + trai + aut + nou =", ali + sem + per + trai + aut + nou)

prod + imp - exp + var             = 20298.0
disp                               = 20298.0
ali + sem + per + trai + aut + nou = 20298.0


# Question 3

In [4]:

# On récupère les csv
animaux = pd.DataFrame(dfs['animal'])
vegetaux = pd.DataFrame(dfs['vegetal'])

# Ajout d'une colonne 'Origine' dans animaux et vegetaux
animaux['Origine'] = 'Animale'
vegetaux['Origine'] = 'Végétale'

# Union de Végétaux et Animaux
aliments = pd.concat([vegetaux, animaux])

# Projection
aliments = aliments[[
    'Pays',
    'Code Pays',
    'Produit',
    'Origine',
    'Élément',
    'Valeur'
]]

# On prend uniquement les éléments qui nous intéresse
kcal = "Disponibilité alimentaire (Kcal/personne/jour)"
prot = "Disponibilité de protéines en quantité (g/personne/jour)"
poid = "Disponibilité alimentaire en quantité (kg/personne/an)"
aliments.query("Élément == @kcal or Élément == @prot or Élément == @poid", inplace=True)

# Table pivot sur aliments
aliments = aliments.pivot_table('Valeur', index=['Code Pays','Pays','Produit','Origine'], columns='Élément')

# Reset index
aliments.reset_index(inplace=True)

# Joindre population et aliments
aliments = pd.merge(aliments, dfs['population'][['Pays', 'Valeur']])

# Renommer les colonnes
aliments.rename(columns={
    'Disponibilité alimentaire (Kcal/personne/jour)'           : 'kcal/pers/an',
    'Disponibilité de protéines en quantité (g/personne/jour)' : 'kg prot/pers/an',
    'Disponibilité alimentaire en quantité (kg/personne/an)'   : 'kg/pers/an',
    'Valeur' : 'population'
}, inplace=True)

# On fait les calculs pour correspondre aux unités
aliments['population'] *= 1000
aliments['kcal/pers/an'] *= 365
aliments['kg prot/pers/an'] *= 365
aliments['kg prot/pers/an'] /= 1000

aliments['énergie/poids'] = aliments['kcal/pers/an'] / aliments['kg/pers/an']
aliments['kg prot/poids'] = aliments['kg prot/pers/an'] / aliments['kg/pers/an']

# Remplacer les Nan et Inf
aliments.replace(np.nan, 0, inplace=True)
aliments.replace(np.inf, 0, inplace=True)

# Affichage
print('Résultat : ', aliments.shape[1], 'colonnes, ', aliments.shape[0], 'lignes')
aliments[['Pays', 'Produit', 'Origine', 'kcal/pers/an', 'kg prot/pers/an', 'population']].sample(10)


Résultat :  10 colonnes,  4405 lignes


,Pays,Produit,Origine,kcal/pers/an,kg prot/pers/an,population
1032,Cuba,Pamplemousse,Végétale,730.0,0.01095,11266000
4026,Burkina Faso,Bière,Végétale,3285.0,0.03650,16935000
1818,Iraq,Cephalopodes,Animale,0.0,0.00000,33765000
780,Sri Lanka,Pois,Végétale,4745.0,0.30660,21273000
1527,Kiribati,"Oranges, Mandarines",Végétale,730.0,0.00000,102000
2185,Lesotho,Pommes de Terre,Végétale,37960.0,0.90520,2074000
2515,Mali,"Plantes Oleiferes, Autre",Végétale,1095.0,0.04745,15302000
1197,France,"Mollusques, Autres",Animale,1460.0,0.21900,64291000
2291,Malawi,Avoine,Végétale,0.0,0.00000,16363000
2701,Nicaragua,Aliments pour enfants,Végétale,5110.0,0.20805,6080000


# Question 4

In [5]:

print("Moyenne du rapport énergie/poids des oeufs (kcal/kg) :")

print(np.mean(aliments.query("Produit == 'Oeufs'")['énergie/poids']))

print()

aliments[['Pays', 'Produit', 'Origine', 'énergie/poids']].sample(10)


Moyenne du rapport énergie/poids des oeufs (kcal/kg) :
1362.3173838894493



,Pays,Produit,Origine,énergie/poids
2545,Mozambique,Arachides Decortiquees,Végétale,5353.333333
647,Cabo Verde,"Beurre, Ghee",Animale,6441.176471
3937,Oman,Viande de Suides,Animale,0.000000
1373,Gabon,Pois,Végétale,0.000000
1840,Iraq,Huile de Soja,Végétale,8942.500000
1020,Cuba,"Légumes, Autres",Végétale,217.208034
635,Canada,Viande de Volailles,Animale,1442.884406
1972,Côte d'Ivoire,"Viande, Autre",Animale,1077.331759
111,Angola,Poivre,Végétale,0.000000
1898,Côte d'Ivoire,Café,Végétale,493.243243


# Question 5 part 1

In [10]:

# On calcul la moyenne de l'énergie sur le poids pour chaque produit, puis on trie dans l'ordre décroissant
prod_en_poids = aliments[['Produit', 'énergie/poids']].groupby('Produit').mean().sort_values('énergie/poids', ascending=False)

prod_en_poids.head(20)


,énergie/poids
Produit,
Huile de Palme,8712.998046
Huile de Soja,8671.981316
Graisses Animales Crue,7714.200488
Huil Plantes Oleif Autr,7497.492849
Huile de Tournesol,7264.736416
"Beurre, Ghee",6607.422927
Huile de Son de Riz,6208.686869
Huile d'Olive,6134.059095
Huile de Coco,6066.931493


# Question 5 part 2

In [11]:

# Pareil mais pour la quantité de protéine
prod_prot_poids = aliments[['Produit', 'kg prot/poids']].groupby('Produit').mean().sort_values('kg prot/poids', ascending=False)

prod_prot_poids.head(20)


,kg prot/poids
Produit,
Arachides Decortiquees,0.245061
Soja,0.230789
Haricots,0.202210
Légumineuses Autres,0.197367
Pois,0.192526
Abats Comestible,0.176632
"Viande, Autre",0.160918
Viande de Bovins,0.146130
Viande d'Ovins/Caprins,0.137785


# Question 6

In [18]:

df = dfs['vegetal'][['Élément', 'Produit', 'Valeur']]

# On fait la somme de la disponibilité intérieur de tous les pays pour chaque produit
df = df.query("Élément == 'Disponibilité intérieure'").groupby('Produit').sum()

# On merge les infos trouvés plus haut
df = pd.merge(df, prod_en_poids, left_index = True, right_index = True)

df.rename(columns={'Valeur' : 'poids (kg)', 'énergie/poids' : 'kcal/kg'}, inplace=True)

# On fait les calculs pour correspondre aux unités
df['poids (kg)'] *= 1000000

# On ajoute la colonne énergie total
df['énergie (kcal)'] = df['poids (kg)'] * df['kcal/kg']

df.head(5)


,poids (kg),kcal/kg,énergie (kcal)
Produit,,,
"Agrumes, Autres",1.672800e+10,73.585330,1.230935e+12
Aliments pour enfants,2.170000e+08,2619.335821,5.683959e+11
Ananas,2.512700e+10,301.483372,7.575373e+12
Arachides Decortiquees,4.116300e+10,5502.020083,2.264797e+14
Avoine,2.416900e+10,1319.777192,3.189769e+13


# Question 7

In [19]:

# La somme de l'énergie de chaque produit
somme_k = df['énergie (kcal)'].sum()

# La consommation moyenne de calorie d'un humain par an
conso_h_k = 2400 * 365

# On refais la même chose que l'exercice précédent mais pour la protéine
df = pd.merge(df, prod_prot_poids, left_index=True, right_index=True)

df['protéine (kg)'] = df['poids (kg)'] * df['kg prot/poids']

# La quantité total de protéine
somme_p = df['protéine (kg)'].sum()

# La consommation moyenne de protéine d'un humain par an
conso_h_p = 0.048 * 365

print("Calorie (kcal)")
print("  Energie total :", somme_k, "kcal")
print("  Energie consommé par un humain en un an :", conso_h_k, "kcal")
print("  Nombre d'humain nourris en calorie :", somme_k / conso_h_k)
print("  Soit", np.round((somme_k / conso_h_k) / nbr_total_humain * 100, 2), "% de la population mondiale")
print()
print("Protéine (kg)")
print("  Protéine total :", somme_p, "kg")
print("  Protéine consommé par un humain en un an :", conso_h_p, "kg")
print("  Nombre d'humain nourris en protéine:", somme_p / conso_h_p)
print("  Soit", np.round((somme_p / conso_h_p) / nbr_total_humain * 100, 2), "% de la population mondiale")

Calorie (kcal)
  Energie total : 1.4239212817891508e+16 kcal
  Energie consommé par un humain en un an : 876000 kcal
  Nombre d'humain nourris en calorie : 16254809152.844187
  Soit 232.3 % de la population mondiale

Protéine (kg)
  Protéine total : 365650679123.62585 kg
  Protéine consommé par un humain en un an : 17.52 kg
  Nombre d'humain nourris en protéine: 20870472552.718372
  Soit 298.26 % de la population mondiale


# Question 8

